In [2]:
# Import Dependencies to web scrape
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser

In [3]:
# Create path from chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# url of page to be scraped
gym_trainer_url = 'https://bulbapedia.bulbagarden.net/wiki/Gym_Leader'
browser.visit(gym_trainer_url)

In [5]:
# set an html object to be used by BeautifulSoup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Grab all the tables of pokemon gym leaders
results = soup.find_all('table', class_='roundy')

# Initialize final league list
leagues = []
# Initialize flag determining new region
region_flag = False
# Initialize counter to go past 'Region' value
counter = 0
# Initialize each gym dict data
gym = {}
    
for result in results:
    league = result.find('th').text.replace("\n",'')
    atags = result.find_all('a')
    for a in atags:
        try:
            # Skip any of these Value Links
            if 'Generation' in a['title'] or 'Badge' in a['title'] or 'Type' in a['title']:
                continue
                                
            # if region_flag is true, we at are a new region
            if counter > 0:
                if '(type)' not in a['title']:
                    # Check to see if its a Location
                    if ('City' in a['title'] or '???' in a['title'] or 
                        'Town' in a['title'] or 'Island' in a['title'] or
                        'Islands' in a['title']
                       ): 
                        gym['city'] = a['title']
                    else:
                        # We are at a Gym Leader
                        # Check to see if Gym Leader exists
                        if 'gym_leader' in gym.keys():
                            # if string, turn it into a list
                            if type(gym['gym_leader']) != list:
                                previous_val = [gym['gym_leader']]
                                previous_val.append(a['title'])
                                gym['gym_leader'] = previous_val
                            else: # just append gym leader to list
                                previous_val = gym['gym_leader']
                                previous_val.append(a['title'])
                                gym['gym_leader'] = previous_val    
                        else:
                            gym['gym_leader'] = a['title']

                if '(type)' in a['title']:
                    # close off all the dictionaries 
                    gym['type'] = a['title']
                    gyms.append(gym)
                    ##print('GYM INFO ', gym)
                    gym = {}
            
            if region_flag:
                # initialize region dictionary & gym
                counter = 1
                gyms_dict = {}
                # set key 
                region = a['title']
                gyms_dict[region] = None
                # initialize list info
                gyms = []
                # reset flag
                region_flag = False
                
                
            # If we find region, start a Flag for the next tag
            if 'Region' == a['title']:
                region_flag = True
                if counter > 0:
                    gyms_dict[region] = gyms
                    leagues.append(gyms_dict)
                    # Reset counter of new region
                    counter = 0
        except KeyError:
            continue


In [6]:
leagues

[{'Kanto': [{'gym_leader': 'Brock',
    'city': 'Pewter City',
    'type': 'Rock (type)'},
   {'gym_leader': 'Misty', 'city': 'Cerulean City', 'type': 'Water (type)'},
   {'gym_leader': 'Lt. Surge',
    'city': 'Vermilion City',
    'type': 'Electric (type)'},
   {'gym_leader': 'Erika', 'city': 'Celadon City', 'type': 'Grass (type)'},
   {'gym_leader': ['Koga', 'Janine'],
    'city': 'Fuchsia City',
    'type': 'Poison (type)'},
   {'gym_leader': 'Sabrina', 'city': 'Saffron City', 'type': 'Psychic (type)'},
   {'gym_leader': 'Blaine', 'city': 'Cinnabar Island', 'type': 'Fire (type)'},
   {'city': 'Viridian City',
    'gym_leader': ['Giovanni', 'Blue (game)'],
    'type': 'Ground (type)'},
   {'type': 'Various (type)'}]},
 {'Johto': [{'gym_leader': ['Johto League', 'Region', 'Falkner'],
    'city': 'Violet City',
    'type': 'Flying (type)'},
   {'gym_leader': 'Bugsy', 'city': 'Azalea Town', 'type': 'Bug (type)'},
   {'gym_leader': 'Whitney',
    'city': 'Goldenrod City',
    'type': 'N